In [2]:
import csv
import math
import pandas as pd
from obspy.taup import TauPyModel
from obspy.core import UTCDateTime
from obspy.geodetics import locations2degrees

class GeoPoint():
    def __init__(self, lon, lat, isRadian=False):
        # self.anomaly = anomaly if anomaly else None
        if isRadian:
            self._longitude = lon
            self._latitude = lat
        else:
            self._longitude = lon / 180 * math.pi
            self._latitude = lat / 180 * math.pi
        self._regulate_angle()
        self.longitude = {"rad": self._get_angle('lon', 'rad'), "deg": self._get_angle('lon', 'deg')}
        self.latitude = {"rad": self._get_angle('lat', 'rad'), "deg": self._get_angle('lat', 'deg')}

    def _get_angle(self, inp: str, oup: str = 'deg'):
        self._regulate_angle()

        if inp == "lon": quantity = self._longitude
        elif inp == "lat": quantity = self._latitude

        if oup == "rad": return float(quantity)
        elif oup == "deg": return quantity / math.pi * 180

    def _regulate_angle(self):
        "regulate longitude within [-pi,*pi) and latitude [-pi/2,*pi/2]"
        self._longitude = (self._longitude+math.pi)%(2*math.pi)-math.pi
        self._latitude = math.acos(math.cos(self._latitude+math.pi/2))-math.pi/2
            
    def midpoint(self, point1, point2):
        # const φ1 = lat1 * Math.PI/180; // φ, λ in radians
        # const φ2 = lat2 * Math.PI/180;
        # const Δφ = (lat2-lat1) * Math.PI/180;
        # const Δλ = (lon2-lon1) * Math.PI/180;
        # const Bx = Math.cos(φ2) * Math.cos(λ2-λ1);
        # const By = Math.cos(φ2) * Math.sin(λ2-λ1);
        # const φ3 = Math.atan2(Math.sin(φ1) + Math.sin(φ2),
        #                     Math.sqrt( (Math.cos(φ1)+Bx)*(Math.cos(φ1)+Bx) + By*By ) );
        # const λ3 = λ1 + Math.atan2(By, Math.cos(φ1) + Bx);
        dLon = point2._longitude - point1._longitude
        Bx = math.cos(point2._latitude) * math.cos(dLon)
        By = math.cos(point2._latitude) * math.sin(dLon)
        mlat = math.atan2(math.sin(point1._latitude) + math.sin(point2._latitude),
                        math.sqrt((math.cos(point1._latitude) + Bx) * (math.cos(point1._latitude) + Bx) + By * By))
        mlon = point1._longitude + math.atan2(By, math.cos(point1._latitude) + Bx)
        return GeoPoint(mlon, mlat, isRadian=True)

In [ ]:
df = pd.read_csv('catalog_2011_preproc.csv')
result = df.loc[(df['source_origin_time'] == str(UTCDateTime("2011-01-05T06:46:18.300Z"))) & (df['receiver_code'] == 'ANMO')]

In [ ]:
result = df.loc[(df['source_origin_time'] == timeref) & (df['receiver_code'] == 'RODM')]
# result = ref_table_slice.loc[(df['source_distance_deg'] == 94.522187)]

In [ ]:
result['source_depth_km']

In [ ]:
import glob
from obspy import read_events, UTCDateTime

# arrival_ref = read_events(f"/Users/jun/waveformget/quakeml/P/2017-01-0*.xml")
# print(dir(arrival_ref[0]))
origin_time_gcmt_to_isc = {}
arrival_db = read_events(); arrival_db = arrival_db[0:0]
for filename in glob.glob(f"/Users/jun/waveformget/quakeml/P/2017-01-0*.xml"):
    new_arrival = read_events(filename)[0]
    origin_time_gcmt_to_isc[filename.split('/')[-1][:-4]] = new_arrival.preferred_origin().time
    arrival_db.append(new_arrival)
    
origin_time = UTCDateTime("2017-01-02T00:13:06.020000Z")
aa = arrival_db.filter(f"time >= {origin_time-1}", f"time <= {origin_time+1}")

In [ ]:
aa[10].preferred_origin()

In [ ]:
origin_gcmt_to_isc = {}
# arrival_db = read_events(); arrival_db = arrival_db[0:0]
year = "updeANMO_shift5_catalog_2010_stnflt_houser_p.csv".split('_')[3]
for filename in glob.glob(f"/Users/jun/waveformget/quakeml/{'p'.upper()}/{year}*.xml"):
    if len(read_events(filename)) > 0:
        new_arrival = read_events(filename)[0]
        origin_gcmt_to_isc[f"{filename.split('/')[-1][:-4]}"] = new_arrival.preferred_origin().time
        # origin_isc_to_gcmt[str(new_arrival.preferred_origin().time)] = filename.split('/')[-1][:-4]
        # arrival_db.append(new_arrival)
print("finished loading isc origin times")

In [9]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
from obspy import read_events, UTCDateTime
from picker import SeismicData, Event, Station
from obspy.geodetics.base import gps2dist_azimuth
from multiprocessing.pool import ThreadPool

class ResultPlot():
    def __init__(self, input_dir="/Users/jun/phasepick/hmsl_pred", quakeml_dir="/Users/jun/waveformget/quakeml", pkl_table=None, csv_table=None):
        table_dir = "/Users/jun/Downloads/drive-download-20220512T014633Z-001"
        self.input_dir = input_dir
        self.quakeml_dir = quakeml_dir
        if pkl_table:
            self.table_filenames = None
            with open(pkl_table, 'rb') as file:
                self.table = pickle.load(file)
        elif csv_table:
            self.table = pd.read_csv(csv_table)
            self.table_filenames = None
        else:
            self.table = None
            self.table_filenames = {"P": f"{table_dir}/Pcomb.4.07.09.table", "S": f"{table_dir}/Scomb.4.07.09.table"}

    def _midpoint(self, evlon, evlat, stlon, stlat):
        return GeoPoint.midpoint(_, GeoPoint(evlon, evlat), GeoPoint(stlon, stlat))

    def dumpall_isc(self, phase, output):
        origin_gcmt_to_isc = {}
        origin_isc_to_gcmt = {}
        # model = TauPyModel(model="prem")
        # model_isc = TauPyModel(model="ak135")

        # build dict for origin time by gcmt for isc
        year = output.split('_')[3]
        # print(year)
        arrival_db = read_events(); arrival_db = arrival_db[0:0]
        for filename in glob.glob(f"/Users/jun/waveformget/quakeml/{phase.upper()}/{year}*.xml"):
            if len(read_events(filename)) > 0:
                new_arrival = read_events(filename)[0]
                origin_gcmt_to_isc[filename.split('/')[-1][:-4]] = new_arrival.preferred_origin().time
                origin_isc_to_gcmt[str(new_arrival.preferred_origin().time)] = filename.split('/')[-1][:-4]
                arrival_db.append(new_arrival)

        def get_event_by_gcmt_origin(origin_time):
            if type(origin_time) is str: origin_time = UTCDateTime(origin_time)
            return arrival_db.filter(f"time >= {origin_time-1}", f"time <= {origin_time+1}")[0]
        
        def get_pick_by_arrival(arrival, picks):
            for pick in picks:
                if pick.resource_id == arrival.pick_id: return pick
        
        with open(output, 'w') as outp:
            writer = csv.writer(outp)
            writer.writerow(['file_name', 'turning_lon', 'turning_lat', 'gcarc_isc', 'origin_depth_km', 'origin_time_gcmt', 'origin_time_isc', f'{phase.lower()}_arrival_ml', f'{phase.lower()}_arrival_isc', f'{phase.lower()}_travel_residue_isc', f'{phase.lower()}_travel_residue_ml_fit_isc', f'{phase.lower()}_probability_ml', f'{phase.lower()}_snr_ml', 'anomaly_org'])
            # print(arrival_db)
            for event in arrival_db:
                isc_origin = event.preferred_origin()
                for arrival in isc_origin.arrivals:
                    isc_pick = get_pick_by_arrival(arrival, event.picks)
                    if isc_pick is None: continue
                    station_code = isc_pick.waveform_id.station_code
                    filename = f"{self.input_dir}/{station_code}_outputs/X_prediction_results.csv"
                    if not os.path.exists(filename): continue

                    ml_picks_by_station = pd.read_csv(filename)
                    station_match = ml_picks_by_station.loc[(ml_picks_by_station['station'] == station_code) | (ml_picks_by_station['station'] == f"{station_code} ")]
                    if len(station_match) == 0: continue
                    network_code = station_match.iloc[0]['network']
                    file_name_format = f"{station_code}_{network_code}_LH_{UTCDateTime(origin_isc_to_gcmt[str(isc_origin.time)], precision=3)}"
                    file_name_match = ml_picks_by_station.loc[ml_picks_by_station['file_name'] == file_name_format]
                    if len(file_name_match) == 0: continue
                    if not file_name_match.iloc[0][f'{phase.lower()}_arrival_time']: continue

                    
                    arrival_ml = UTCDateTime(f"{file_name_match.iloc[0][f'{phase.lower()}_arrival_time'].replace(' ', 'T')}Z")
                    station_lat = file_name_match.iloc[0]['station_lat']
                    station_lon = file_name_match.iloc[0]['station_lon']
                    calctim_isc_rebuilt = (isc_pick.time - isc_origin.time) - arrival.time_residual # calctim = traveltime - traveltime_residue
                    time_residue_ml = (arrival_ml - isc_origin.time) - calctim_isc_rebuilt # eqiv arrival_ml - arrival_isc + ttres
                    

                    point = self._midpoint(isc_origin.longitude, isc_origin.latitude, station_lon, station_lat)
                    writer.writerow([
                        file_name_format, #file_name
                        '%.3f'%point.longitude['deg'], #turning_lon
                        '%.3f'%point.latitude['deg'], #turning_lat
                        # '%.4f'%gcarc_gcmt, #gcarc_gcmt
                        '%.3f'%arrival.distance, #gcarc_isc
                        '%.3f'%(isc_origin.depth/1000.0), #origin_depth_km
                        origin_isc_to_gcmt[str(isc_origin.time)], #origin_time_gcmt
                        isc_origin.time, #origin_time_isc
                        arrival_ml, #arrival_time_ml
                        isc_pick.time, #arrival_time_isc
                        '%.4f'%arrival.time_residual, #travel_time_residue_isc
                        '%.4f'%time_residue_ml, #travel_time_residue_ml_fit_isc
                        file_name_match.iloc[0][f'{phase.lower()}_probability'], #probability_ml
                        file_name_match.iloc[0][f'{phase.lower()}_snr'], #snr_ml
                        None, #anomaly_org, only used to compare HMSL-06
                        ])


    # def dumpall(self, phase, output):
    #     # data = []
    #     model = TauPyModel(model="prem")
    #     model_isc = TauPyModel(model="ak135")

    #     # for filename in glob.glob(f"{self.quakeml_dir}/{phase.upper()}/*.xml"):
    #     #     origin_gcmt_to_isc[str(UTCDateTime((filename.split('/')[-1])[:-4]))] = str(UTCDateTime(read_events(filename)[0].origins[0].time))

    #     # build dict for origin time by gcmt for isc
    #     origin_gcmt_to_isc = {}
    #     origin_isc_to_gcmt = {}
    #     year = output.split('_')[3]
    #     arrival_db = read_events(); arrival_db = arrival_db[0:0]
    #     for filename in glob.glob(f"/Users/jun/waveformget/quakeml/{phase.upper()}/{year}*.xml"):
    #         if len(read_events(filename)) > 0:
    #             new_arrival = read_events(filename)[0]
    #             origin_gcmt_to_isc[filename.split('/')[-1][:-4]] = new_arrival.preferred_origin().time
    #             origin_isc_to_gcmt[str(new_arrival.preferred_origin().time)] = filename.split('/')[-1][:-4]
    #             arrival_db.append(new_arrival)

    #     # def get_event_by_gcmt_origin(origin_time):
    #     #     if type(origin_time) is str: origin_time = UTCDateTime(origin_time)
    #     #     return arrival_db.filter(f"time >= {origin_time-1}", f"time <= {origin_time+1}")[0]
        
    #     # create table and save
    #     with open(output, 'w') as outp:
    #         writer = csv.writer(outp)
    #         fileanames = glob.glob(f"{self.input_dir}/*_outputs/X_prediction_results.csv")
    #         # arrival_ref = read_events(f"{self.quakeml_dir}/{phase.upper()}/*.xml")
    #         writer.writerow(['file_name', 'turning_lon', 'turning_lat', 'gcarc_gcmt', 'gcarc_isc', 'anomaly', 'probability', 'anomaly_org', 'calctim_ak135' ,'anomaly_ak135', 'anomaly_isc'])
    #         for fileaname in fileanames:
    #             table_sta = pd.read_csv(fileaname)
    #             print(f"processing {fileaname}")
    #             for index, record in table_sta.iterrows():
    #                 timestamp_rec = UTCDateTime(record['file_name'].split('_')[-1])
    #                 records_found = self.table.loc[(self.table['source_origin_time'] == str(timestamp_rec)) & (self.table['receiver_code'] == record['station'].rstrip())]
    #                 if len(records_found) == 0: continue # ignore the predictions coming from waveform files that failed sanity check as training data
    #                 if not type(record[f'{phase.lower()}_arrival_time']) == str: continue

    #                 # isc_origin = event.preferred_origin()

    #                 # arrival_ref = read_events(f"{self.quakeml_dir}/{phase.upper()}/{timestamp_rec}.xml")
    #                 record_match = records_found.iloc[0]
    #                 gcarc_gcmt = record_match['source_distance_deg']
    #                 gcarc_isc = None


    #                 # gcarc_isc = arrival_ref
    #                 # arrival_time_isc = arrival_ref
    #                 # calctim_gcmt_prem = None
    #                 # calctim_isc_ak135 = None
                    
                    
                    
    #                 # is_p_calculated = ('p_calculated_travel_sec' in self.table.columns) & (phase.lower() == 'p')
    #                 # calctim = [record_match['p_calculated_travel_sec']] if is_p_calculated else model.get_travel_times(record_match['source_depth_km'], gcarc, [phase])
    #                 calc_gcmt_prem = model.get_travel_times(record_match['source_depth_km'], gcarc_gcmt, [phase])
    #                 if len(calc_gcmt_prem) == 0: continue
    #                 else: calctim_gcmt_prem = calctim_gcmt_prem[0].time
    #                 calc_isc_ak135 = model_isc.get_travel_times(record_match['source_depth_km'], gcarc_gcmt, [phase])
    #                 if len(calc_isc_ak135) != 0: calctim_isc_ak135 = calc_isc_ak135[0].time

    #                 eqlon = record_match['source_longitude']
    #                 eqlat = record_match['source_latitude']
    #                 anomaly_org = None

    #                 if abs(anomaly) < 20:
    #                     point = self._midpoint(eqlon, eqlat, record['station_lon'], record['station_lat'])
    #                     # writer.writerow([record['file_name'],'%.3f'%point.longitude['deg'], '%.3f'%point.latitude['deg'], '%.4f'%gcarc, '%.4f'%anomaly, record[f'{phase.lower()}_probability'], '%.4f'%anomaly_org if anomaly_org else None])
    #                     # data.append([
    #                     writer.writerow([
    #                         record['file_name'], #file_name
    #                         '%.3f'%point.longitude['deg'], #turning_lon
    #                         '%.3f'%point.latitude['deg'], #turning_lat
    #                         '%.4f'%gcarc_gcmt, #gcarc_gcmt
    #                         '%.4f'%gcarc_isc, #gcarc_isc
    #                         #origin_time_gcmt
    #                         #origin_time_isc
    #                         #arrival_time_ml
    #                         #arrival_time_isc
    #                         '%.4f'%anomaly, #travel_time_ml
    #                         #travel_time_isc
    #                         '%.4f'%calctim_gcmt_prem, # calculated travel time using `gcmt` origin and `prem` velocity model
    #                         '%.4f'%calctim_isc_ak135, # calculated travel time using `isc` origin and `ak135` velocity model
    #                         record[f'{phase.lower()}_probability'], #probability_ml
    #                         '%.4f'%anomaly_org if anomaly_org else None, #anomaly_org, only used to compare HMSL-06
    #                         # calctim_ak135, 
    #                         # anomaly_ak135, 
    #                         # anomaly_isc
    #                         ])


    #         # writer.writerows(data)

    def find_isc(self, year):
        self.origin_gcmt_to_isc = {}
        for filename in glob.glob(f"/Users/jun/waveformget/quakeml/P/{year}*.xml"):
            events_read = read_events(filename)
            if len(events_read) > 0:
                self.origin_gcmt_to_isc[f"{filename.split('/')[-1][:-4]}"] = events_read[0].preferred_origin().time

        for filename in glob.glob(f"/Users/jun/waveformget/quakeml/S/{year}*.xml"):
            if not f"{filename.split('/')[-1][:-4]}" in self.origin_gcmt_to_isc:
                events_read = read_events(filename)
                if len(events_read) > 0:
                    self.origin_gcmt_to_isc[f"{filename.split('/')[-1][:-4]}"] = events_read[0].preferred_origin().time
        print("finished loading isc origin times")

    def add_origin_hmsl_to_gcmt(self, phase):
        table_file = f"/Users/jun/Downloads/drive-download-20220512T014633Z-001/{phase}comb.4.07.09.table"
        self.origin_hmsl_to_gcmt = {}
        table = pd.read_csv(table_file, delim_whitespace=True)
        for index, record in table.iterrows():
            # print(record['year'], record['day'], record['hour'], record['min'], record['sec'])
            hmsl_timestamp = UTCDateTime(year=int(record['year']), julday=int(record['day']),
                hour=int(record['hour']), minute=int(record['min']),
                second=int(f"{min(record['sec'],59.999)}".split('.')[0]), microsecond=int(f"{min(record['sec'],59.999)}".split('.')[1]))
            if not str(hmsl_timestamp) in self.origin_hmsl_to_gcmt:
                gcmt_timestamp = None
                candidates = [ UTCDateTime(search_timestamp.split('/')[-1][:27]) for search_timestamp in glob.glob(f"/Users/jun/waveformget/quakeml_hmsl/{phase}/{hmsl_timestamp.year}-{hmsl_timestamp.month:02d}-{hmsl_timestamp.day:02d}*")]
                if len(candidates) > 0:
                    gcmt_timestamp = candidates[0]
                    for candidate in candidates:
                        if abs(candidate-hmsl_timestamp) < abs(gcmt_timestamp-hmsl_timestamp):
                            gcmt_timestamp = candidate
                self.origin_hmsl_to_gcmt[str(hmsl_timestamp)] = gcmt_timestamp

    def dump_hmsl(self, phase, output):
        table_file = f"/Users/jun/Downloads/drive-download-20220512T014633Z-001/{phase}comb.4.07.09.table"
        table = pd.read_csv(table_file, delim_whitespace=True)
        with open(output, 'w') as outp:
            writer = csv.writer(outp)
            writer.writerow(['file_name', 'turning_lon', 'turning_lat', 'gcarc_isc', 'origin_depth_km', 'origin_time_gcmt', 'origin_time_isc', f'{phase.lower()}_arrival_ml', f'{phase.lower()}_arrival_isc', f'{phase.lower()}_travel_residue_isc', f'{phase.lower()}_travel_residue_ml_fit_isc', f'{phase.lower()}_probability_ml', f'{phase.lower()}_snr_ml', 'anomaly_org'])
                
            for index, record in table.iterrows():
                # print(record['year'], record['day'], record['hour'], record['min'], record['sec'])
                hmsl_timestamp = UTCDateTime(year=int(record['year']), julday=int(record['day']),
                    hour=int(record['hour']), minute=int(record['min']),
                    second=int(f"{min(record['sec'],59.999)}".split('.')[0]), microsecond=int(f"{min(record['sec'],59.999)}".split('.')[1]))
                if not str(hmsl_timestamp) in self.origin_hmsl_to_gcmt: continue
                if self.origin_hmsl_to_gcmt[str(hmsl_timestamp)] is None: continue

                def get_pick_by_arrival(arrival, picks):
                    for pick in picks:
                        if pick.resource_id == arrival.pick_id: return pick
            

                filename = f"/Users/jun/waveformget/quakeml_hmsl/{phase}/{self.origin_hmsl_to_gcmt[str(hmsl_timestamp)]}.xml"
                events_read = read_events(filename)
                if len(events_read) == 0: continue
                events_read = events_read[0]
                isc_origin = events_read.preferred_origin()
                for arrival in isc_origin.arrivals:
                        isc_pick = get_pick_by_arrival(arrival, events_read.picks)
                        if isc_pick is None: continue
                        station_code = isc_pick.waveform_id.station_code
                        if station_code != record['station']: continue
                        network_code = isc_pick.waveform_id.network_code

                        hmsl_residue = record['obstim'] -  record['calctim'] + record['ellipcor']
                        # station_match = ml_picks_by_station.loc[(ml_picks_by_station['station'] == station_code) | (ml_picks_by_station['station'] == f"{station_code} ")]
                        # if len(station_match) == 0: continue
                        # network_code = station_match.iloc[0]['network']
                        file_name_format = f"{station_code}_{network_code}_LH_{UTCDateTime(str(hmsl_timestamp), precision=3)}"
                        # file_name_match = ml_picks_by_station.loc[ml_picks_by_station['file_name'] == file_name_format]
                        # if len(file_name_match) == 0: continue
                        # if not file_name_match.iloc[0][f'{phase.lower()}_arrival_time']: continue

                        
                        arrival_ml = UTCDateTime(isc_pick.time - arrival.time_residual + hmsl_residue) # eqiv arrival_ml - arrival_isc + ttres
                        station_lat = record['stalat']
                        station_lon = record['stalon']
                        calctim_isc_rebuilt = (isc_pick.time - isc_origin.time) - arrival.time_residual # calctim = traveltime - traveltime_residue
                        time_residue_ml = hmsl_residue
                        

                        point = self._midpoint(isc_origin.longitude, isc_origin.latitude, station_lon, station_lat)
                        writer.writerow([
                            file_name_format, #file_name
                            '%.3f'%point.longitude['deg'], #turning_lon
                            '%.3f'%point.latitude['deg'], #turning_lat
                            # '%.4f'%gcarc_gcmt, #gcarc_gcmt
                            '%.3f'%arrival.distance, #gcarc_isc
                            '%.3f'%(isc_origin.depth/1000.0), #origin_depth_km
                            hmsl_timestamp, #origin_time_gcmt
                            isc_origin.time, #origin_time_isc
                            arrival_ml, #arrival_time_ml
                            isc_pick.time, #arrival_time_isc
                            '%.4f'%arrival.time_residual, #travel_time_residue_isc
                            '%.4f'%time_residue_ml, #travel_time_residue_ml_fit_isc
                            1, #probability_ml
                            20, #snr_ml
                            None, #anomaly_org, only used to compare HMSL-06
                            ])

    # for takumi, us array
    def dumpall_houser(self, phase, output):

        model = TauPyModel(model="ak135")
        with open(output, 'w') as outp:
            writer = csv.writer(outp, delimiter=' ', quoting=csv.QUOTE_MINIMAL, quotechar='"',escapechar=' ')
            if self.table_filenames:
                table_org = pd.read_csv(self.table_filenames[phase], delim_whitespace=True) 
                table_alt = pd.read_csv(self.table_filenames['S' if phase=='P' else 'P'], delim_whitespace=True)
            else:
                table_org = None
                table_alt = None

            fileanames = glob.glob(f"{self.input_dir}/*_outputs/X_prediction_results.csv")
            # writer.writerow(['file_name', 'turning_lon', 'turning_lat', 'gcarc' , 'anomaly', 'probability', 'anomaly_org'])
            writer.writerow([
                '%8s'%'eqlat',
                '%8s'%'eqlon',
                '%6s'%'eqdep',
                '%8s'%'stalat',
                '%8s'%'stalon',
                '%8s'%'distdeg',
                '%8s'%'azimuth',
                '%10s'%'predttime',
                '%10s'%'obsvttime',
                '%5s'%'prob',
                f'{"filename":<37}'
            ])

            for fileaname in fileanames:
                table_sta = pd.read_csv(fileaname)
                print(f"processing {fileaname}")
                for index, record in table_sta.iterrows():
                    timestamp_rec = UTCDateTime(record['file_name'].split('_')[-1], precision=6)
                    if int(timestamp_rec.year) != int(year): continue
                    if not str(timestamp_rec) in self.origin_gcmt_to_isc: continue
                    isc_origin = self.origin_gcmt_to_isc[str(timestamp_rec)]
                    records_found = self.table.loc[(self.table['source_origin_time'] == str(timestamp_rec)) & (self.table['receiver_code'] == record['station'].rstrip())]
                    if len(records_found) == 0: continue # ignore the predictions coming from waveform files that failed sanity check as training data
                    record_match = records_found.iloc[0]
                    if len(str(record[f'{phase.lower()}_arrival_time']))<10: continue
                    # if phase.lower() == 'p':
                    #     calctim = record_match[f'{phase.lower()}_calculate_travel_sec']
                    #     if np.isnan(calctim): continue
                    # else:
                    if True:
                        calctim = model.get_travel_times(record_match['source_depth_km'], record_match['source_distance_deg'], [phase])
                        if len(calctim) == 0: continue
                        calctim = calctim[0].time
                        azimuth = gps2dist_azimuth(lat1=record_match['source_latitude'], lon1=record_match['source_longitude'],
                                                   lat2=record_match['receiver_latitude'], lon2=record_match['receiver_longitude'])[1]


                    # gcarc = record_match['source_distance_deg']
                    # calctim = record_match[f'{phase.lower()}_calculate_travel_sec']
                    # eqlon = record_match['source_longitude']
                    # eqlat = record_match['source_latitude']
                    # eqdep = record_match['source_depth_km']
                    # eqlon = record_match['receiver_longitude']
                    # eqlat = record_match['receiver_latitude']
                    # traveltime = record[f'{phase.lower()}_arrival_time'] - timestamp_rec
                    # file_name = record['file_name'] 
                    
                    # print(record[f'{phase.lower()}_arrival_time'])
                    writer.writerow([
                        f"{record_match['source_latitude']:>8.2f}", #source_lat
                        f"{record_match['source_longitude']:>8.2f}", #source_lon
                        f"{record_match['source_depth_km']:>6.1f}", #source_dep_km
                        f"{record_match['receiver_latitude']:>8.2f}", #receiver_lat
                        f"{record_match['receiver_longitude']:>8.2f}", #receiver_lon
                        f"{record_match['source_distance_deg']:>8.3f}", #gcarc_gcmt
                        f"{azimuth:>8.3f}", #azimuth
                        f"{calctim:>10.4f}", #predicted_traveltime
                        f"{(UTCDateTime(record[f'{phase.lower()}_arrival_time']) - isc_origin):>10.4f}", #observed_traveltime
                        f"{record[f'{phase.lower()}_probability']}", #phase_probability
                        f"{record['file_name']:<37}", #file_name
                    ])

    def dumpall(self, phase, output):
        data = []
        model = TauPyModel(model="prem")
        with open(output, 'w') as outp:
            writer = csv.writer(outp)
            if self.table_filenames:
                table_org = pd.read_csv(self.table_filenames[phase], delim_whitespace=True) 
                table_alt = pd.read_csv(self.table_filenames['S' if phase=='P' else 'P'], delim_whitespace=True)
            else:
                table_org = None
                table_alt = None
            fileanames = glob.glob(f"{self.input_dir}/*_outputs/X_prediction_results.csv")
            # writer.writerow(['file_name', 'turning_lon', 'turning_lat', 'gcarc' , 'anomaly', 'probability', 'anomaly_org'])
            for fileaname in fileanames:
                table_sta = pd.read_csv(fileaname)
                print(f"processing {fileaname}")
                for index, record in table_sta.iterrows():
                    if type(record[f'{phase.lower()}_arrival_time']) == str:
                        timestamp_rec = UTCDateTime(record['file_name'].split('_')[-1])
                        if self.table is None: # using HMSL tables as reference
                            try:
                                table_match = table_org[(table_org['station']==record['station'].rstrip()) & (table_org['year']==int(timestamp_rec.year)) & (table_org['day']==int(timestamp_rec.julday)) & (table_org['hour']==int(timestamp_rec.hour)) & (table_org['min']==int(timestamp_rec.minute)) & (abs(table_org['sec']-int(timestamp_rec.second))<2)].to_dict(orient='records')[0]
                                anomaly_org = table_match['obstim'] - table_match['calctim']
                                anomaly = UTCDateTime(record[f'{phase.lower()}_arrival_time']) - timestamp_rec - table_match['calctim']
                                gcarc = locations2degrees(table_match['eqlat'], table_match['eqlon'], table_match['stalat'], table_match['stalon'])
                            except:
                                table_match = table_alt[(table_alt['station']==record['station'].rstrip()) & (table_alt['year']==int(timestamp_rec.year)) & (table_alt['day']==int(timestamp_rec.julday)) & (table_alt['hour']==int(timestamp_rec.hour)) & (table_alt['min']==int(timestamp_rec.minute)) & (abs(table_alt['sec']-int(timestamp_rec.second))<2)].to_dict(orient='records')[0]
                                anomaly_org = None
                                gcarc = locations2degrees(table_match['eqlat'], table_match['eqlon'], table_match['stalat'], table_match['stalon'])
                                calctim = model.get_travel_times(table_match['eqdep'], gcarc, [phase])
                                if len(calctim) > 0:
                                    anomaly = UTCDateTime(record[f'{phase.lower()}_arrival_time']) - timestamp_rec - calctim[0].time
                            eqlon = table_match['eqlon']
                            eqlat = table_match['eqlat']
                        else:
                            if type(self.table) == pd.DataFrame:
                                # print(self.table)
                                records_found = self.table.loc[(self.table['source_origin_time'] == str(timestamp_rec)) & (self.table['receiver_code'] == record['station'].rstrip())]
                                if len(records_found) == 0: continue # ignore the predictions coming from waveform files that failed sanity check as training data
                                record_match = records_found.iloc[0]
                                gcarc = record_match['source_distance_deg']
                                calctim = model.get_travel_times(record_match['source_depth_km'], gcarc, [phase])
                                eqlon = record_match['source_longitude']
                                eqlat = record_match['source_latitude']
                                anomaly_org = None
                                if len(calctim) > 0:
                                    anomaly = UTCDateTime(record[f'{phase.lower()}_arrival_time']) - timestamp_rec - calctim[0].time
                                else: continue

                            else: 
                                # using event-pair information from pickle file
                                event_match = None
                                station_match = None
                                anomaly_org = None
                                for event in self.table.events:
                                    if event.srctime == timestamp_rec: event_match = event; break
                                for station in event_match.stations:
                                    if station.labelsta['name'] == record['station'].rstrip(): station_match = station; break
                                if station_match is None: continue

                                gcarc = station_match.labelsta['dist']
                                calctim = model.get_travel_times(event_match.srcloc[2], gcarc, [phase])
                                eqlon = event_match.srcloc[1]
                                eqlat = event_match.srcloc[0]
                                if len(calctim) > 0:
                                    anomaly = UTCDateTime(record[f'{phase.lower()}_arrival_time']) - timestamp_rec - calctim[0].time
                                else: continue

                        if abs(anomaly) < 20:
                            point = self._midpoint(eqlon, eqlat, record['station_lon'], record['station_lat'])
                            # writer.writerow([record['file_name'],'%.3f'%point.longitude['deg'], '%.3f'%point.latitude['deg'], '%.4f'%gcarc, '%.4f'%anomaly, record[f'{phase.lower()}_probability'], '%.4f'%anomaly_org if anomaly_org else None])
                            data.append([record['file_name'],'%.3f'%point.longitude['deg'], '%.3f'%point.latitude['deg'], '%.4f'%gcarc, '%.4f'%anomaly, record[f'{phase.lower()}_probability'], '%.4f'%anomaly_org if anomaly_org else None])


            writer.writerows(data)

In [ ]:
folder_prefix = f"catalog_2015_stnflt"
plot = ResultPlot(input_dir=f"./updeANMO_shift5_pred_{folder_prefix}", csv_table=f'./{folder_prefix[:12]}_preproc.csv')
plot.add_origin_hmsl_to_gcmt("P")
plot.add_origin_hmsl_to_gcmt("S")


In [ ]:
tmp = plot.origin_hmsl_to_gcmt
plot = ResultPlot(input_dir=f"./updeANMO_shift5_pred_{folder_prefix}", csv_table=f'./{folder_prefix[:12]}_preproc.csv')
plot.origin_hmsl_to_gcmt = tmp

In [ ]:
plot.dump_hmsl("S", "hmsl_isc_plot_s.csv")

In [ ]:
plot.dump_hmsl("P", "hmsl_isc_plot_p.csv")

In [10]:
year = 2010
folder_prefix = f"catalog_usta_{year}"
plot = ResultPlot(input_dir=f"./updeANMO_shift5_pred_{folder_prefix}", csv_table=f'./{folder_prefix}_preproc.csv')
plot.find_isc(int(folder_prefix.split('_')[-1]))
# plot.dumpall_houser('P', f'./updeANMO_shift5_{folder_prefix}_houser_p.csv')
plot.dumpall_houser('S', f'./updeANMO_shift5_{folder_prefix}_houser_s.csv')

KeyboardInterrupt: 

In [11]:
# year = 2011
for year in range(2010,2012+5):
    folder_prefix = f"catalog_usta_{year}"
    plot = ResultPlot(input_dir=f"./updeANMO_shift5_pred_{folder_prefix}", csv_table=f'./{folder_prefix}_preproc.csv')
    plot.find_isc(int(folder_prefix.split('_')[-1]))
    plot.dumpall_houser('P', f'./updeANMO_shift5_{folder_prefix}_houser_p_prob.csv')
    plot.dumpall_houser('S', f'./updeANMO_shift5_{folder_prefix}_houser_s_prob.csv')

finished loading isc origin times
processing ./updeANMO_shift5_pred_catalog_usta_2010/L04D_outputs/X_prediction_results.csv
processing ./updeANMO_shift5_pred_catalog_usta_2010/EGAK_outputs/X_prediction_results.csv
processing ./updeANMO_shift5_pred_catalog_usta_2010/109C_outputs/X_prediction_results.csv
processing ./updeANMO_shift5_pred_catalog_usta_2010/I35A_outputs/X_prediction_results.csv
processing ./updeANMO_shift5_pred_catalog_usta_2010/228A_outputs/X_prediction_results.csv
processing ./updeANMO_shift5_pred_catalog_usta_2010/U24A_outputs/X_prediction_results.csv
processing ./updeANMO_shift5_pred_catalog_usta_2010/I34A_outputs/X_prediction_results.csv
processing ./updeANMO_shift5_pred_catalog_usta_2010/229A_outputs/X_prediction_results.csv
processing ./updeANMO_shift5_pred_catalog_usta_2010/U25A_outputs/X_prediction_results.csv
processing ./updeANMO_shift5_pred_catalog_usta_2010/E37A_outputs/X_prediction_results.csv
processing ./updeANMO_shift5_pred_catalog_usta_2010/Y26A_outputs/X

FileNotFoundError: [Errno 2] No such file or directory: './catalog_usta_2015_preproc.csv'

In [ ]:
start_year = 2015
num_years = 2

folder_prefix_list = [f"catalog_{start_year+year}_stnflt" for year in range(num_years)]
for folder_prefix in folder_prefix_list:
    # folder_name =  f"{folder_prefix}_hdfs"
    plot = ResultPlot(input_dir=f"./updeANMO_shift5_pred_{folder_prefix}", csv_table=f'./{folder_prefix[:12]}_preproc.csv')
    # plot.dumpall_isc('P', f'./updeANMO_shift5_{folder_prefix}_isc_plot_p.csv')
    # plot.dumpall_isc('S', f'./updeANMO_shift5_{folder_prefix}_isc_plot_s.csv')
    # plot.dumpall('P', f'./updeANMO_shift5_{folder_prefix}_plot_p.csv')
    # plot.dumpall('S', f'./updeANMO_shift5_{folder_prefix}_plot_s.csv')
    plot.find_isc(int(folder_prefix.split('_')[1]))
    plot.dumpall_houser('P', f'./updeANMO_shift5_{folder_prefix}_houser_p.csv')
    plot.dumpall_houser('S', f'./updeANMO_shift5_{folder_prefix}_houser_s.csv')

In [ ]:
plot = ResultPlot(input_dir="updeANMO_shift5_pred_catalog3", pkl_table='./rawdata_catalog3/data_fetched_catalog_2010_3.pkl')
plot.dumpall('P', './updeANMO_shift5_catalog_2010_stnflt_plot_p.csv')
plot.dumpall('S', './updeANMO_shift5_catalog_2010_stnflt_plot_s.csv')

In [ ]:
df2 = pd.Index(['May', 'Jun', 'Jul', 'Aug'])

In [ ]:
df.columns.append(pd.Index(['filename']))

In [ ]:
import glob
import numpy as np
import pandas as pd
from cmcrameri import cm
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

# plt.figure(figsize=(24,8))
# plt.rcParams["font.size"] = 16
# # plt.style.use('dark_background')
# # map = Basemap(projection='ortho', 
# #               lat_0=0, lon_0=0)
# map = Basemap(projection='moll',lon_0=-180,resolution='c')
# map.drawmapboundary(fill_color='white')
# # map.drawmapboundary(fill_color='aqua')
# # map.fillcontinents(color='coral',lake_color='aqua')

def fidelity_rev(picker1_prob, picker1_anomaly, picker2_prob, picker2_anomaly): return picker1_prob - 0.1#0.72
def fidelity_rev_p(picker1_prob, picker1_anomaly, picker2_prob, picker2_anomaly): return picker1_prob - 0.5#0.75
# def fidelity_rev(picker1_prob, picker1_anomaly, picker2_prob, picker2_anomaly): return picker1_prob - 0.7#0.72
# def fidelity_rev_p(picker1_prob, picker1_anomaly, picker2_prob, picker2_anomaly): return picker1_prob - 0.9#0.75
def fidelity_rev_p_s(picker1_prob, picker1_anomaly, picker2_prob, picker2_anomaly): return (((picker1_prob - 0.5)+abs(picker1_prob - 0.5)/2) + ((picker2_prob - 0.1)+abs(picker2_prob - 0.1)/2))#0.75

def plot_depthslice(phase: str, value: str, gcarc_range: set, fidelity_func, org_constraint=lambda tb: pd.isnull(tb) | pd.notnull(tb), raw_prefix='updeANMO_shift_plot', ref_prefix=None, mark_size=lambda r: 50*r**2, demean=False, average_ray=False, plot_legend=False, plot_colorbar=True, plot=True):
    if not '-' in phase: # simple phase arrival
        table_raw = pd.concat([pd.read_csv(filename) for filename in glob.glob(f'{raw_prefix}_{phase.lower()}.csv')], ignore_index=True)
        if ref_prefix:
            table_ref = pd.read_csv(f'{ref_prefix}_{phase.lower()}.csv')
            columns_to_add = pd.DataFrame({'file_name': table_ref['file_name'].values, 'probability_ref': table_ref['probability'].values, 'anomaly_ref': table_ref['anomaly'].values})
            table = table_raw.merge(columns_to_add, how='left', on='file_name')
        else: table = table_raw
    else: # differential phase arrival
        table_raw = pd.concat([pd.read_csv(filename) for filename in glob.glob(f'{raw_prefix}_{phase.split("-")[0].lower()}.csv')], ignore_index=True)
        table_ref = pd.concat([pd.read_csv(filename) for filename in glob.glob(f'{raw_prefix}_{phase.split("-")[1].lower()}.csv')], ignore_index=True)
        columns_to_add = table_ref.columns.difference(table_raw.columns).append(pd.Index(['file_name']))
        table = table_raw.merge(table_ref[columns_to_add], how='left', on='file_name')

    if f'{phase.split("-")[0]}_travel_residue_ml_fit_isc' in table.columns:
        if not '-' in phase: phase1 = phase.lower(); phase2 = phase.lower()
        else: phase1 = phase.split("-")[0].lower(); phase2 = phase.split("-")[1].lower()
        picker1_prob = table[f'{phase1}_probability_ml']
        picker1_anomaly = table[f'{phase1}_travel_residue_ml_fit_isc']
        picker2_prob = table[f'{phase2}_probability_ml']
        picker2_anomaly = table[f'{phase2}_travel_residue_isc']
        gcarc = table['gcarc_isc']
    else:
        picker1_prob = table['probability']
        picker1_anomaly = table['anomaly']
        picker2_prob = table['probability']
        picker2_anomaly = table['anomaly']
        gcarc = table['gcarc']

    fidelity = fidelity_func(picker1_prob=picker1_prob,
                             picker1_anomaly=picker1_anomaly,
                             picker2_prob=picker2_prob,
                             picker2_anomaly=picker2_anomaly)
    if value=='anomaly_org': fidelity = (fidelity+1)
    scatter_table = table[(org_constraint(table['anomaly_org'])) & (fidelity > 0) & (gcarc > gcarc_range[0]) & (gcarc < gcarc_range[1])]
    if average_ray:
        count_org = len(scatter_table)
        # print(f"Before: {count_org}")
        std = np.std(scatter_table[value].values[:])
        for idx, rec in scatter_table.iterrows():
            half_length = 2
            while (True):
                neighbor_value = scatter_table.loc[(scatter_table['turning_lon']<rec['turning_lon']+half_length) & (scatter_table['turning_lon']>rec['turning_lon']-half_length) & (scatter_table['turning_lat']<rec['turning_lat']+half_length) & (scatter_table['turning_lat']>rec['turning_lat']-half_length), value]
                if len(neighbor_value) > 20: half_length /= 1.5
                else: break
            if len(neighbor_value) > 3:
                if (rec[value] > np.mean(neighbor_value) + std) or (rec[value] < np.mean(neighbor_value) - std):
                    # print(rec.name, inx)
                    table.loc[idx, picker1_prob.name] = 0
        scatter_table = table[(org_constraint(table['anomaly_org'])) & (picker1_prob > 0) & (fidelity > 0) & (gcarc > gcarc_range[0]) & (gcarc < gcarc_range[1])]
        print(f"{count_org-len(scatter_table)} outliers are removed")
        # print(f"After: len(scatter_table)")

    lons = scatter_table['turning_lon'].values[:]
    lats = scatter_table['turning_lat'].values[:]
    prob = scatter_table[picker1_prob.name].values[:]
    if not '-' in phase: # simple phase arrival
        value_column = scatter_table[value]
        anomaly = value_column.values[:]
        if demean: anomaly -= np.nanmean(anomaly)
    else: # differential phase arrival
        value_column = scatter_table[f'{phase.split("-")[0].lower()}_travel_residue_ml_fit_isc'] - scatter_table[f'{phase.split("-")[1].lower()}_travel_residue_ml_fit_isc']
        anomaly = value_column.values[:]
    print(f"{sum(value_column.notnull())} points fit the conditions")
    # print(type(anomaly[0]))
    if not plot: return scatter_table

    map = Basemap(projection='moll',lon_0=-180,resolution='c')
    x, y = map(lons, lats)
    # print(np.array(mark_size(prob), dtype=float))

    map.drawmapboundary(fill_color='white')
    map.drawcoastlines()
    # map.scatter(x, y)
    map.scatter(x, y, c=anomaly, s=mark_size(prob), marker='o', cmap=cm.vik)
    # map.scatter(x, y, c=anomaly, marker='o', cmap=cm.vik)
    colorscale = 15 if phase.lower() == 's' else 8
    plt.clim(-colorscale, colorscale)
    plt.title(f'{phase.upper()} travel time residuals (sec) in {gcarc_range[0]}~{gcarc_range[1]} deg\n{"demeaned" if demean else""} $\delta t{f"_{phase.upper()}" if value=="anomaly" else ""}$, {sum(value_column.notnull())} pts')

    if plot_colorbar:
        map.colorbar(location='bottom')

    if plot_legend:
        sizes = [0.2, 0.4, 0.6, 0.8, 1.0]
        markers = [plt.scatter([], [], s=mark_size(size), edgecolors='none', color='dimgray') for size in sizes]
        labels = [f'{size:.1f}' for size in sizes]
        plt.legend(markers, labels, title="probability", scatterpoints=1, frameon=False, loc='lower left', bbox_to_anchor=(0.97, 0.75), fontsize=12)

    return scatter_table

top_gcarc = 0
gcarc_interval = 200

# plt.subplot(121)
p_table = plot_depthslice('p', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev_p, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot=False)
# # plot_depthslice('p', f'p_travel_residue_ml_fit_isc', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev_p, raw_prefix='updeANMO_shift5_catalog_*_stnflt_isc_plot', demean=True)
# # plot_depthslice('s-p', 'p-s', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev_p_s, raw_prefix='updeANMO_shift5_catalog_*_stnflt_isc_plot', demean=True)
# plt.subplot(122)
# # plot_depthslice('s', f's_travel_residue_ml_fit_isc', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_*_stnflt_isc_plot', demean=True)
s_table = plot_depthslice('s', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot=False)
# # plot_depthslice('s', 'anomaly', (90,110), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_plot')
# plt.subplots_adjust(wspace=0.1)
# plt.show()

In [ ]:
fig = plt.figure(figsize=(24,8))
plt.rcParams["font.size"] = 16
map = Basemap(projection='moll',lon_0=-180,resolution='c')

top_gcarc = 70
gcarc_interval = 10

def fidelity_rev(picker1_prob, picker1_anomaly, picker2_prob, picker2_anomaly): return picker1_prob - 0.1#0.72
def fidelity_rev_p(picker1_prob, picker1_anomaly, picker2_prob, picker2_anomaly): return picker1_prob - 0.5#0.75

plt.subplot(121)
p_table = plot_depthslice('p', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev_p, raw_prefix='updeANMO_shift5_plot', average_ray=False, demean=True, plot_legend=True)
plt.subplot(122)
s_table = plot_depthslice('s', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev, raw_prefix='updeANMO_shift5_plot', average_ray=False, demean=True)
plt.subplots_adjust(wspace=0.1)
plt.show()

In [ ]:
from sklearn.linear_model import RANSACRegressor, TheilSenRegressor

def fidelity_rev(picker1_prob, picker1_anomaly, picker2_prob, picker2_anomaly): return picker1_prob - 0.7#0.72
def fidelity_rev_p(picker1_prob, picker1_anomaly, picker2_prob, picker2_anomaly): return picker1_prob - 0.9#0.75

top_gcarc = 30
gcarc_interval = 10

plt.rcParams["font.size"] = 12
plt.figure(figsize=(12,24))

def plot_ratio(gcarc_range, plot_xlabel=False, plot_ylabel=False):
    p_table = plot_depthslice('p', 'anomaly', gcarc_range, fidelity_rev_p, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', demean=True, average_ray=True, plot=False)
    s_table = plot_depthslice('s', 'anomaly', gcarc_range, fidelity_rev, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', demean=True, average_ray=True, plot=False)
    p_table = p_table.rename(columns={'anomaly': 'p_anomaly', 'probability': 'p_probability'})
    s_table = s_table.rename(columns={'anomaly': 's_anomaly', 'probability': 's_probability'})
    columns_to_add = s_table.columns.difference(p_table.columns).append(pd.Index(['file_name']))
    scatter_table = p_table.merge(s_table[columns_to_add], how='left', on='file_name')
    scatter_table = scatter_table[scatter_table['s_probability']>0.1]

    x = scatter_table['p_anomaly']
    y = scatter_table['s_anomaly']
    x_prob = scatter_table['p_probability']
    y_prob = scatter_table['s_probability']
    # x -= np.mean(x); y -= np.mean(y)
    plt.scatter(x.values, y.values, s=15)
    # xseq = np.linspace(-10, 10, num=100)
    xseq = np.linspace(min(x.values), max(x.values), num=100)

    # b, a = np.polyfit(x.values, y.values, deg=1)
    # # b, a = np.polyfit(x.values, y.values, w=(x_prob.values+y_prob.values)/2, deg=1)
    # plt.plot(xseq, a + b * xseq, color="k", lw=2)

    # a = np.polyfit(x.values, y.values, deg=4)
    # plt.plot(xseq, a[0] * xseq**3 + a[1] * xseq ** 2 + a[2] * xseq + a[3], color="k", lw=3)

    theilsen = TheilSenRegressor(random_state=42).fit(y.values.reshape(-1,1), x.values)
    y_bound = np.array([min(y.values), max(y.values)])
    # y_bound = np.array([-10,10]).reshape(-1,1)
    b = round(float(np.diff(y_bound) / np.diff(theilsen.predict(y_bound.reshape(-1,1)))), 4)
    plt.plot(theilsen.predict(y_bound.reshape(-1,1)), y_bound, color="k", lw=2)

    plt.xlim([-20,20]); plt.ylim([-20,20])
    if plot_xlabel: plt.xlabel("$\delta t_P (sec)$")
    if plot_ylabel: plt.ylabel("$\delta t_S (sec)$")
    plt.title(f'P and S travel time residuals (sec) in {top_gcarc}~{top_gcarc+gcarc_interval} deg\n#Point = {len(x.values)}, $\delta T_S/ \delta T_P$ = {"%.4f"%b}')

plt.cla(); plt.clf()
plt.subplot(421); plot_ratio((top_gcarc, top_gcarc+gcarc_interval),plot_ylabel=True); top_gcarc+=gcarc_interval
plt.subplot(423); plot_ratio((top_gcarc, top_gcarc+gcarc_interval),plot_ylabel=True); top_gcarc+=gcarc_interval
plt.subplot(425); plot_ratio((top_gcarc, top_gcarc+gcarc_interval),plot_ylabel=True); top_gcarc+=gcarc_interval
plt.subplot(427); plot_ratio((top_gcarc, top_gcarc+gcarc_interval),plot_ylabel=True,plot_xlabel=True); top_gcarc+=gcarc_interval
plt.subplot(422); plot_ratio((top_gcarc, top_gcarc+gcarc_interval)); top_gcarc+=gcarc_interval
plt.subplot(424); plot_ratio((top_gcarc, top_gcarc+gcarc_interval)); top_gcarc+=gcarc_interval
plt.subplot(426); plot_ratio((top_gcarc, top_gcarc+gcarc_interval),plot_xlabel=True); top_gcarc+=gcarc_interval
plt.subplots_adjust(hspace=0.2)
plt.show()

In [ ]:
# print(427 + 1003 + 733 + 1663 + 888 + 1987 + 1378 + 2406 + 1692 + 2184 + 1883 + 1950 + 740 + 323)
# print(4093 + 12897 + 7025 + 21465 + 8441 + 24652 + 11916 + 24917 + 15105 + 26065 + 16036 + 20067 + 6501 + 5511)
# print(19257 + 204691)
print(30, round(6371 - 5595.9), round(6371 - 5577.8))
print(40, round(6371 - 5409.4), round(6371 - 5427.9))
print(50, round(6371 - 5134.8), round(6371 - 5193.5))
print(60, round(6371 - 4813.6), round(6371 - 4903.6))
print(70, round(6371 - 4458.8), round(6371 - 4575.5))
print(80, round(6371 - 4062.2), round(6371 - 4207.1))
print(90, round(6371 - 3629.4), round(6371 - 3796.8))
print(100, round(6371 - 3490.7), round(6371 - 3544.0))

In [ ]:
top_gcarc = 30
gcarc_interval = 10

plt.figure(figsize=(24,28))
plt.rcParams["font.size"] = 16
plt.subplot(421); plot_depthslice('p', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev_p, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_legend=True, plot_colorbar=False)
plt.subplot(422); plot_depthslice('s', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=False)
top_gcarc += gcarc_interval
plt.subplot(423); plot_depthslice('p', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev_p, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=False)
plt.subplot(424); plot_depthslice('s', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=False)
top_gcarc += gcarc_interval
plt.subplot(425); plot_depthslice('p', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev_p, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=False)
plt.subplot(426); plot_depthslice('s', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=False)
top_gcarc += gcarc_interval
plt.subplot(427); plot_depthslice('p', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev_p, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=True)
plt.subplot(428); plot_depthslice('s', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=True)
# plot_depthslice('s', 'anomaly', (90,110), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_plot')
plt.subplots_adjust(wspace=0.1)
plt.show()

In [ ]:
top_gcarc = 70
gcarc_interval = 10

plt.figure(figsize=(24,21))
plt.rcParams["font.size"] = 16
plt.subplot(321); plot_depthslice('p', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev_p, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_legend=True, plot_colorbar=False)
plt.subplot(322); plot_depthslice('s', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=False)
top_gcarc += gcarc_interval
plt.subplot(323); plot_depthslice('p', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev_p, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=False)
plt.subplot(324); plot_depthslice('s', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=False)
top_gcarc += gcarc_interval
plt.subplot(325); plot_depthslice('p', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev_p, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=True)
plt.subplot(326); plot_depthslice('s', 'anomaly', (top_gcarc,top_gcarc+gcarc_interval), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', average_ray=False, plot_colorbar=True)
# plot_depthslice('s', 'anomaly', (90,110), fidelity_rev, raw_prefix='updeANMO_shift5_catalog_plot')
plt.subplots_adjust(wspace=0.1)
plt.show()

In [ ]:
import glob
import numpy as np
import pandas as pd
from cmcrameri import cm
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

def fidelity_rev(picker1_prob, picker1_anomaly, picker2_prob, picker2_anomaly): return picker1_prob - 0.7#0.72
def fidelity_rev_p(picker1_prob, picker1_anomaly, picker2_prob, picker2_anomaly): return picker1_prob - 0.9#0.75

def plot_ratio_depthslice(gcarc_range, ratio, plot_colorbar=False, plot_legend=False, mark_size=lambda r: 50*r**2):
    p_table = plot_depthslice('p', 'anomaly', gcarc_range, fidelity_rev_p, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', demean=True, average_ray=True, plot=False)
    s_table = plot_depthslice('s', 'anomaly', gcarc_range, fidelity_rev, raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot', demean=True, average_ray=True, plot=False)
    p_table = p_table.rename(columns={'anomaly': 'p_anomaly', 'probability': 'p_probability'})
    s_table = s_table.rename(columns={'anomaly': 's_anomaly', 'probability': 's_probability'})
    columns_to_add = s_table.columns.difference(p_table.columns).append(pd.Index(['file_name']))
    scatter_table = p_table.merge(s_table[columns_to_add], how='left', on='file_name')
    scatter_table = scatter_table[(scatter_table['s_probability']>0.1)]

    lons = scatter_table['turning_lon'].values[:]
    lats = scatter_table['turning_lat'].values[:]
    prob = np.amin([scatter_table['p_probability'].values[:], scatter_table['p_probability'].values[:]], axis=0)
    # value_column = scatter_table['s_anomaly'] / scatter_table['p_anomaly']
    dts = scatter_table['s_anomaly']; dtp = scatter_table['p_anomaly']
    # value_column = (dts - np.mean(dts.values))/np.std(dts.values) - (dtp - np.mean(dtp.values))/np.std(dtp.values)
    value_column = (dts - np.mean(dts.values))/ratio - (dtp - np.mean(dtp.values))
    # anomaly = np.arctan(value_column.values[:])
    anomaly = (value_column.values[:])
    print(f"{sum(value_column.notnull())} points fit the conditions")
    # print(type(anomaly[0]))
    # if not plot: return scatter_table

    x, y = map(lons, lats)
    # print(np.array(mark_size(prob), dtype=float))

    map.drawmapboundary(fill_color='white')
    map.drawcoastlines()
    # map.scatter(x, y)
    map.scatter(x, y, c=anomaly, s=mark_size(prob), marker='o', cmap=cm.broc)
    # map.scatter(x, y, c=anomaly, marker='o', cmap=cm.vik)
    # colorscale = 15 if phase.lower() == 's' else 8
    # plt.clim(-colorscale, colorscale)
    plt.clim(-4, 4)
    plt.title(f'Travel time residuals difference in {gcarc_range[0]}~{gcarc_range[1]} deg\n demeaned $\delta t_S/{ratio:.4f} -$ demeaned $\delta t_P$, {sum(value_column.notnull())} pts')

    if plot_colorbar:
        map.colorbar(location='bottom')

    if plot_legend:
        sizes = [0.2, 0.4, 0.6, 0.8, 1.0]
        markers = [plt.scatter([], [], s=mark_size(size), edgecolors='none', color='dimgray') for size in sizes]
        labels = [f'{size:.1f}' for size in sizes]
        plt.legend(markers, labels, title="probability", scatterpoints=1, frameon=False, loc='lower left', bbox_to_anchor=(0.97, 0.75), fontsize=12)

    return scatter_table

top_gcarc = 30
gcarc_interval = 10

plt.cla(); plt.clf()
plt.rcParams["font.size"] = 16
plt.figure(figsize=(24,28))
map = Basemap(projection='moll',lon_0=-180,resolution='c')

plt.subplot(421); plot_ratio_depthslice((top_gcarc, top_gcarc+gcarc_interval),ratio=2.4269, plot_legend=True); top_gcarc+=gcarc_interval
plt.subplot(423); plot_ratio_depthslice((top_gcarc, top_gcarc+gcarc_interval),ratio=2.4436); top_gcarc+=gcarc_interval
plt.subplot(425); plot_ratio_depthslice((top_gcarc, top_gcarc+gcarc_interval),ratio=2.6537); top_gcarc+=gcarc_interval
plt.subplot(427); plot_ratio_depthslice((top_gcarc, top_gcarc+gcarc_interval),ratio=2.6960, plot_colorbar=True); top_gcarc+=gcarc_interval
plt.subplot(422); plot_ratio_depthslice((top_gcarc, top_gcarc+gcarc_interval),ratio=2.4969); top_gcarc+=gcarc_interval
plt.subplot(424); plot_ratio_depthslice((top_gcarc, top_gcarc+gcarc_interval),ratio=2.8419); top_gcarc+=gcarc_interval
plt.subplot(426); plot_ratio_depthslice((top_gcarc, top_gcarc+gcarc_interval),ratio=3.2559, plot_colorbar=True); top_gcarc+=gcarc_interval
plt.subplots_adjust(wspace=0.1)
# plot_ratio_depthslice((top_gcarc, top_gcarc+gcarc_interval),plot_colorbar=True)
plt.show()

In [ ]:
np.polyfit(x.values, y.values, deg=4)

In [ ]:
y.values

In [ ]:
raw_prefix = "updeANMO_shift5_catalog_*_stnflt_plot"
phase = "s"
df = pd.concat([pd.read_csv(filename) for filename in glob.glob(f'{raw_prefix}_{phase.lower()}.csv')], ignore_index=True)
df

In [ ]:
p_table

In [ ]:
# plt.subplots(121)

# p_table.loc[cond,'anomaly'].hist(bins=40).plot()

# plt.subplots(122)
# plt.xlabel("$\delta t_P$ (sec)")
# s_table.loc[cond,'anomaly'].hist(bins=80).plot()
# plt.show()

plt.figure(figsize=(7,5))
cond = (p_table['probability'] > 0.5)
print(p_table.loc[cond,'anomaly'].describe())
p_table.loc[cond,'anomaly'].hist(bins=40, color="blue", grid=True, label='P', alpha=0.6)
plt.ylabel('frequency')
plt.xlabel("$\delta t$ (sec)")
plt.title('distribution of predicted travel time residuals')

cond = (s_table['probability'] > 0.1)
print(s_table.loc[cond,'anomaly'].describe())
plt.hist(s_table.loc[cond,'anomaly'], bins=40, color="red", label='S', alpha=0.6)
plt.legend()
plt.xlim([-20,20])
# plt.savefig("pandas&plt_hist.png")
plt.show()

In [ ]:
import os
import pandas as pd
from obspy import read_inventory, UTCDateTime

station_dict = {}
event_list = []
for idx, val in df['file_name'].items():
    file_name_splits = val.split('_')
    key = f'{file_name_splits[1]}.{file_name_splits[0]}'
    if not key in station_dict: station_dict[key] = file_name_splits[-1]
    if not file_name_splits[-1] in event_list: event_list.append(file_name_splits[-1])

st_lons = []
st_lats = []
df_year = None
for key, event in station_dict.items():
    if df_year != event[:4]:
        df_ref = pd.read_csv(f"./catalog_{event[:4]}_preproc.csv")
        df_year = event[:4]
    found = df_ref.loc[df_ref['receiver_code'] == key.split('.')[-1]].iloc[0]
    st_lons.append(found['receiver_longitude'])
    st_lats.append(found['receiver_latitude'])

ev_lons = []
ev_lats = []
df_year = None
for event in event_list:
    if df_year != event[:4]:
        df_ref = pd.read_csv(f"./catalog_{event[:4]}_preproc.csv")
        df_year = event[:4]
    found = df_ref.loc[df_ref['source_origin_time'] == str(UTCDateTime(event, precision=6))].iloc[0]
    ev_lons.append(found['source_longitude'])
    ev_lats.append(found['source_latitude'])

In [ ]:
df_ref = pd.read_csv(f"./catalog_2014_preproc.csv")
df_ref['p_calculate_travel_sec'].iloc[2] > 0

In [ ]:
None is not None

In [ ]:
import glob
import numpy as np
import pandas as pd
from cmcrameri import cm
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

plt.figure(figsize=(12,8))
plt.rcParams["font.size"] = 16
map = Basemap(projection='moll',lon_0=-180,resolution='c')
# map.drawmapboundary(fill_color='white')

def plot_source_receiver(raw_prefix='updeANMO_shift_plot'):
    table_raw = pd.concat([pd.read_csv(filename) for filename in glob.glob(f'{raw_prefix}_{phase.lower()}.csv')], ignore_index=True)
    
    station_dict = {}
    event_list = []
    for idx, val in df['file_name'].items():
        file_name_splits = val.split('_')
        key = f'{file_name_splits[1]}.{file_name_splits[0]}'
        if not key in station_dict: station_dict[key] = file_name_splits[-1]
        if not file_name_splits[-1] in event_list: event_list.append(file_name_splits[-1])

    st_lons = []
    st_lats = []
    df_year = None
    for key, event in station_dict.items():
        if df_year != event[:4]:
            df_ref = pd.read_csv(f"./catalog_{event[:4]}_preproc.csv")
            df_year = event[:4]
        found = df_ref.loc[df_ref['receiver_code'] == key.split('.')[-1]].iloc[0]
        st_lons.append(found['receiver_longitude'])
        st_lats.append(found['receiver_latitude'])

    ev_lons = []
    ev_lats = []
    df_year = None
    for event in event_list:
        if df_year != event[:4]:
            df_ref = pd.read_csv(f"./catalog_{event[:4]}_preproc.csv")
            df_year = event[:4]
        found = df_ref.loc[df_ref['source_origin_time'] == str(UTCDateTime(event, precision=6))].iloc[0]
        ev_lons.append(found['source_longitude'])
        ev_lats.append(found['source_latitude'])

    map.drawmapboundary(fill_color='white')
    map.drawcoastlines()

    x, y = map(st_lons, st_lats)
    map.scatter(x, y, c='g', marker='v')
    x, y = map(ev_lons, ev_lats)
    map.scatter(x, y, c='r', marker='*')
    # map.scatter(x, y, c=anomaly, marker='o', cmap=cm.vik)
    plt.title(f'source-receiver distribution')

plot_source_receiver(raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot')
plt.show()

In [ ]:
import glob
import numpy as np
import pandas as pd
from cmcrameri import cm
from obspy import UTCDateTime
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

# phase ='p'
plt.rcParams["font.size"] = 16
fig, ax = plt.subplots(figsize=(12,8))
map = Basemap(projection='moll',lon_0=-180,resolution='c',ax=ax)
# map.drawmapboundary(fill_color='white')

# raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot'
def plot_source_receiver(raw_prefix='updeANMO_shift_plot'):
    p_table = pd.concat([pd.read_csv(filename) for filename in glob.glob(f'{raw_prefix}_p.csv')], ignore_index=True)
    s_table = pd.concat([pd.read_csv(filename) for filename in glob.glob(f'{raw_prefix}_s.csv')], ignore_index=True)
    columns_to_add = s_table.columns.difference(p_table.columns).append(pd.Index(['file_name']))
    table = p_table.merge(s_table[columns_to_add], how='outer', on='file_name')

    station_dict = {}
    network_list = []
    event_list = []
    for idx, val in table['file_name'].items():
        file_name_splits = val.split('_')
        key = f'{file_name_splits[1]}.{file_name_splits[0]}'
        if not key in station_dict: station_dict[key] = file_name_splits[-1]
        if not file_name_splits[1] in network_list: network_list.append(file_name_splits[1])
        if not file_name_splits[-1] in event_list: event_list.append(file_name_splits[-1])
    network_list.sort()

    st_lons = []
    st_lats = []
    df_year = None
    for key, event in station_dict.items():
        if df_year != event[:4]:
            df_ref = pd.read_csv(f"./catalog_{event[:4]}_preproc.csv")
            df_year = event[:4]
        found = df_ref.loc[df_ref['receiver_code'] == key.split('.')[-1]].iloc[0]
        st_lons.append(found['receiver_longitude'])
        st_lats.append(found['receiver_latitude'])

    ev_lons = []
    ev_lats = []
    df_year = None
    for event in event_list:
        if df_year != event[:4]:
            df_ref = pd.read_csv(f"./catalog_{event[:4]}_preproc.csv")
            df_year = event[:4]
        found = df_ref.loc[df_ref['source_origin_time'] == str(UTCDateTime(event, precision=6))].iloc[0]
        ev_lons.append(found['source_longitude'])
        ev_lats.append(found['source_latitude'])

    x, y = map(st_lons, st_lats)
    ax.scatter(x, y, c='g', marker='v')
    print(f"{len(st_lons)} stations from network: {network_list}")
    x, y = map(ev_lons, ev_lats)
    ax.scatter(x, y, c='r', marker='*')
    print(f"{len(ev_lons)} events")
    # map.scatter(x, y, c=anomaly, marker='o', cmap=cm.vik)

    map.drawmapboundary(fill_color='white')
    map.drawcoastlines()
    ax.legend(['station', 'earthquake'], loc='lower left', bbox_to_anchor=(0.85, 0.9))
    plt.title(f'source-receiver distribution')

plot_source_receiver(raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot')
plt.show()

In [ ]:
a=['II', 'IU', 'G', 'GE', 'DK', 'PS']
a.sort()
a

In [ ]:
import glob
import pandas as pd
raw_prefix='updeANMO_shift5_catalog_*_stnflt_plot'
table_raw = pd.concat([pd.read_csv(filename) for filename in glob.glob(f'{raw_prefix}_p.csv')], ignore_index=True)
table_ref = pd.concat([pd.read_csv(filename) for filename in glob.glob(f'{raw_prefix}_s.csv')], ignore_index=True)
table_raw = table_raw.rename(columns={'anomaly': 'p_anomaly', 'probability': 'p_probability'})
table_ref = table_ref.rename(columns={'anomaly': 's_anomaly', 'probability': 's_probability'})
columns_to_add = table_ref.columns.difference(table_raw.columns).append(pd.Index(['file_name']))
table = table_raw.merge(table_ref[columns_to_add], how='left', on='file_name')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
top_gcarc = 80
gcarc_interval = 10
gcarc_range = (top_gcarc, top_gcarc+gcarc_interval)
xcorr = lambda table: (table['p_probability'] > 0.7) & (table['s_probability'] > 0.5) & (table['gcarc'] > gcarc_range[0]) & (table['gcarc'] < gcarc_range[1])

scatter_table = table[xcorr(table)]
for value in ['p_anomaly', 's_anomaly']:
    prob_name = f"{value.split('_')[0]}_probability"
    count_org = len(scatter_table)
    std = np.std(scatter_table[value].values[:])
    for idx, rec in scatter_table.iterrows():
        half_length = 2
        while (True):
            neighbor_value = scatter_table.loc[(scatter_table['turning_lon']<rec['turning_lon']+half_length) & (scatter_table['turning_lon']>rec['turning_lon']-half_length) & (scatter_table['turning_lat']<rec['turning_lat']+half_length) & (scatter_table['turning_lat']>rec['turning_lat']-half_length), value]
            if len(neighbor_value) > 20: half_length /= 1.5
            else: break
        if len(neighbor_value) > 3:
            if (rec[value] > np.mean(neighbor_value) + std) or (rec[value] < np.mean(neighbor_value) - std):
                # print(rec.name, inx)
                table.loc[idx, prob_name] = 0
    scatter_table = table[xcorr(table)]
    print(f"{count_org-len(scatter_table)} outliers are removed")

In [ ]:
# scatter_table = table[xcorr]
x = table.loc[xcorr, 'p_anomaly']
y = table.loc[xcorr, 's_anomaly']
plt.scatter(x.values, y.values, s=0.5)
b, a = np.polyfit(x.values, y.values, deg=1)
xseq = np.linspace(min(x.values), max(x.values), num=100)
plt.plot(xseq, a + b * xseq, color="k", lw=3)
plt.xlabel(x.name)
plt.ylabel(y.name)
plt.title(f'P/S ratio between gcarc distance {gcarc_range}\n#Point = {len(x.values)}, slope = {"%.4f"%b}')
plt.show()


In [ ]:
from obspy.taup import TauPyModel
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(9, 9))
plt.title("source depth: 500 km, distance: 92, 132 deg")
ax.axis('off')
model = TauPyModel(model="prem")
arrivals = model.get_ray_paths(source_depth_in_km=500, distance_in_degree=92, phase_list=["P", "PP", "PcP", "PS", "SP", "ScS", "SS", "S"])
ax = arrivals.plot_rays(fig=fig, legend=True)
# plt.)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
plt.title("source depth: 500 km, distance: 92, 132 deg")
ax.axis('off')
model = TauPyModel(model="prem")
arrivals = model.get_ray_paths(source_depth_in_km=500, distance_in_degree=132, phase_list=["Pdiff", "PP","PS", "SP", "SS", "Sdiff"])
ax = arrivals.plot_rays(fig=fig, legend=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from obspy.taup import plot_travel_times
plt.cla(); plt.clf()
fig, ax = plt.subplots(figsize=(9, 9))
ax = plot_travel_times(source_depth=10, phase_list=["P", "PP", "PcP", "PS", "SP", "ScS", "SS", "S"],
                       ax=ax, fig=fig)

In [ ]:
from obspy.taup import plot_travel_times
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(9, 9))
ax = plot_travel_times(source_depth=10, phase_list=["P", "S", "PP"],
                      ax=ax, fig=fig)